In [1]:
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pylab as plt
import seaborn as sns

In [ ]:
"D:\Đại học\Năm 3\Kỳ 1\Phân tích và trực quan dữ liệu\Đồ án\Data"

In [46]:
%%writefile app.py
import pandas as pd
import numpy as np
import streamlit as st
import time
import matplotlib.pylab as plt
import seaborn as sns
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split



st.set_page_config(layout = "wide")
#st.set_option('deprecation.showPyplotGlobalUse', False)

st.title("DASHBOARD TRỰC QUAN HOÁ DỮ LIỆU")

# Đọc dữ liệu
data = pd.read_csv(r"D:\Đại học\Năm 3\Kỳ 1\Phân tích và trực quan dữ liệu\Đồ án\Data\patient-characteristics-survey-pcs-2017-1.csv")
mota_data = pd.read_excel(r"D:\Đại học\Năm 3\Kỳ 1\Phân tích và trực quan dữ liệu\Đồ án\Data\MOTA_patient.xlsx")

# Chọn thao tác
with st.sidebar:
    select = st.radio(
        "Chọn thao tác",
        options=["Trực quan hoá dữ liệu", "Chạy mô hình", "Bảng mô tả thuộc tính"])
# Thao tác trực quan hoá dữ liệu
if select == "Trực quan hoá dữ liệu":
    st.header("Trực quan hoá dữ liệu")

    # Mô tả về bộ dữ liệu
    st.text_area(label="Sơ lược về bộ dữ liệu", height= 200,
                value=
"""- Tên bộ dữ liệu: "Patient Characteristics Survey (PCS): 2017"\n
- Nguồn tải: https://data.world/data-ny-gov/8itk-gcdy\n
- Kích thước dữ liệu: {0} dòng, {1} cột\n
- Số giá trị null: {2}""".format(data.shape[0], data.shape[1], data.isnull().sum().sum()))

    # Hiển thị 5 dòng ngẫu nhiên
    st.text("Năm dòng ngẫu nhiên từ bộ dữ liệu")
    st.dataframe(data.sample(5))

    # Chọn thuộc tính để vẽ biểu đồ
    columns = st.multiselect("Chọn thuộc tính vẽ biểu đồ", options=data.columns)
    if st.button("Plot"):
        for col in columns:
            # Hiển thị tên cột
            st.text("Thuộc tính: " + col)

            #Set figsize khác nhau cho thuộc tính
            if col == 'Three Digit Residence Zip Code':
                fig, ax = plt.subplots(figsize=(12, 10))
            else:
                fig, ax = plt.subplots(figsize=(10, 4))
            
            # Vẽ biểu đồ
            sns.countplot(y=col, data=data)
            st.pyplot(fig)
            
            #Hiển thị số lượng giá trị của từng cột
            value_count = pd.DataFrame({"Giá trị" : data[col].value_counts().index, "Số lượng" : data[col].value_counts().values})
            value_count.set_index("Giá trị")
            st.text("Giá trị và số lượng tương ứng")
            st.dataframe(value_count.T)
            st.text("==============================================================================================================================")
    
    # Tiền xử lí dữ liệu tự động
    st.header("Tiền xử lí dữ liệu")
    st.text_area(label="Các bước tiền xử lí được tự động thực hiện", height= 250,
                value=
"""- Thay thế các giá trị UNKNOWN hoặc tương tự thành Nan\n
- Xoá cột có số giá trị Nan lớn hơn 70%\n
- Xoá tất cả dòng chứa giá trị Nan\n
- Xoá cột chỉ có một giá trị sau khi xử lí\n
- Chuyển các thuộc tính chữ thành số bằng Label Encoder
""")
    unk = ["UNKNOWN",
       "CLIENT DIDN'T ANSWER", 
       "CLIENT DID NOT ANSWER", 
       "UNKNOWN RACE", 
       "UNKNOWN EMPLOYMENT STATUS", 
       "NOT APPLICABLE", 
       "NOT APPLICABLE",
       "UNKNOWN EMPLOYMENT HOURS"]
    # Thay thế giá trị UNKNOWN hoặc tương tự thành Nan
    data.replace(to_replace=unk, value=np.nan, inplace=True)

    # Loại bỏ các cột có trên 70% Nan
    for col in data.columns:
        if data[col].isnull().sum() > 0.7*len(data):
            data.drop(col, axis=1, inplace=True)

    # Bỏ tất cả các dòng chứa giá trị Nan
    data.dropna(inplace=True)

    # Bỏ các cột chỉ chứa một giá trị sau khi xử lí
    for col in data.columns:
        if len(data[col].unique()) <= 1:
            data.drop(col, axis=1, inplace=True)
    # Chuyển các thuộc tính chữ thành số bằng Label Encoder
    label_encoder = preprocessing.LabelEncoder()
    for col in data.columns:
        if data[col].dtypes != "int64":
            data[col]= label_encoder.fit_transform(data[col])
    
    # Chuyển cột Stroke về cuối DF
    cols = list(data.columns.values)
    cols.pop(cols.index('Stroke'))
    data = data[cols+['Stroke']]

    st.text("Kích thước dữ liệu sau khi xử lí: {0} dòng, {1} cột".format(data.shape[0], data.shape[1]))
    st.text("Giá trị và số lượng giá trị biến Stroke: " + "NO " + str(data["Stroke"].value_counts()[0]) + ", YES " + str(data["Stroke"].value_counts()[1]))
    # Hiển thị 5 dòng ngẫu nhiên
    st.text("Năm dòng ngẫu nhiên từ bộ dữ liệu")
    st.dataframe(data.sample(5))
    
    # Xem tổng quan data
    if st.checkbox("Xem tổng quan data"):
        #3fig, ax = plt.subplots(figsize=(30, 30))
        data.hist(figsize=(30, 30))
        st.pyplot()

    # Lấy chỉ số tương quan, vẽ heatmap
    st.header("Các thuộc tính ảnh hưởng đến biến Stroke")
    f = st.slider('Chọn hệ số tương quan tối thiểu',0.0, 1.0)
    #Lấy các biến có chỉ số tương quan lớn hơn f
    data_corr = data.corr()
    correlation_index = data_corr[((data_corr["Stroke"] > f) | (data_corr["Stroke"] < -f))]["Stroke"].index
    st.text("Số lượng thuộc tính có hệ số tương quan cao hơn {0} là: ".format(f) + str(len(correlation_index)))

    # Vẽ heatmap theo f
    if st.button("Vẽ heatmap"):
        st.subheader("Heatmap")   
        plt.figure(figsize=(13, 5))
        sns.heatmap(data[correlation_index].corr(), annot=True)
        st.pyplot()

    # Cân bằng dữ liệu
    st.header("Cân bằng dữ liệu")
    st.text_area(label= "Các bước cân bằng dữ liệu", height=100,
                value=
    """- Cân bằng dữ liệu theo cột Stroke\n
- Sử dụng thư viện SMOTE có sẵn""")
    ss = st.slider('Chọn tỉ lệ cân bằng dữ liệu',0.0, 1.0)
    # Lấy X, y từ data
    X = data.drop("Stroke", axis = 1)
    y = data["Stroke"]
    # Cân bằng dữ liệu sử dụng SMOTE
    data_res_smt = pd.DataFrame()
    correlation_index_SMT = []
    #if st.button("Cân bằng"):
    smt = SMOTE(sampling_strategy= ss)
    X_res_smt, y_res_smt = smt.fit_resample(X, y)
    # Gán lại cột Stroke để có data hoàn chỉnh
    data_res_smt = X_res_smt
    data_res_smt["Stroke"] = y_res_smt
    st.text("Giá trị và số lượng giá trị biến Stroke: " + "NO " + str(data_res_smt["Stroke"].value_counts()[0]) + ", YES " + str(data_res_smt["Stroke"].value_counts()[1]))
    # Xem tổng quan data
    st.dataframe(data_res_smt)
    
    if st.checkbox("Xem tổng quan data sau khi cân bằng"):
        data_res_smt.hist(figsize=(30, 30))
        st.pyplot()
    # Vẽ lại heatmap cho data cân bằng
    st.subheader("Heatmap")
    data_res_smt_corr = data_res_smt.corr()
    f2 = st.slider('Chọn hệ số tương quan tối thiểu ',0.0, 1.0)
    if st.button("Vẽ heatmap "):
    #Lấy các biến có chỉ số tương quan lớn hơn f
        data_res_smt_corr = data_res_smt.corr()
        correlation_index_SMT = data_res_smt_corr[((data_res_smt_corr["Stroke"] > f2) | (data_res_smt_corr["Stroke"] < -f2))]["Stroke"].index
        st.text("Số lượng thuộc tính có hệ số tương quan cao hơn {0} là: ".format(f2) + str(len(correlation_index_SMT)))
        # Vẽ heatmap theo f2
        st.dataframe(data_res_smt[correlation_index_SMT])
        plt.figure(figsize=(13, 5))
        sns.heatmap(data_res_smt[correlation_index_SMT].corr(), annot=True)
        st.pyplot()
# Chọn chức năng xem bảng thuộc tính
elif select == "Bảng mô tả thuộc tính":
    st.header("Bảng mô tả thuộc tính")
    st.dataframe(mota_data)

# Chọn chức năng chạy model trên các loại data
elif select == "Chạy mô hình":
    st.header("Chọn model")

    # Chọn các thuộc tính cần thiết
    model_name = st.selectbox("Chọn model", ("Logistic Regression", "Support Vector Machine", "Naive Bayes", "RandomForest"))
    data_name = st.selectbox("Chọn data", ("Data gốc", "Data xử lí biến UNKNOWN và cân bằng dữ liệu", "Data xử lí biến UNKNOWN, cân bằng dữ liệu và chọn lọc thuộc tính"))
    train_size = st.slider('Chọn train size ',0.0, 1.0)

    # Xử lí các thuộc tính
    with st.spinner('Wait for it...'):
        label_encoder = preprocessing.LabelEncoder()
        # Xử lí model
        if model_name == "Logistic Regression":
            model = LogisticRegression()
        elif model_name == "Support Vector Machine":
            model = SVC()
        elif model_name == "Naive Bayes":
            model = GaussianNB()
        else:
            model_name = RandomForestClassifier(max_depth=10)
        data = pd.read_csv(r"D:\Đại học\Năm 3\Kỳ 1\Phân tích và trực quan dữ liệu\Đồ án\Data\patient-characteristics-survey-pcs-2017-1.csv")

        # Xử lí data gốc
        if data_name == "Data gốc":
            st.text_area(label= "Mô tả bộ dữ liệu", height=50,
            value=
    """- Dữ liệu gốc chỉ thực hiện Label Encoder, có {0} dòng và {1} cột""".format(data.shape[0], data.shape[1]))

            for col in data.columns:
                if data[col].dtypes != "int64":
                    data[col]= label_encoder.fit_transform(data[col])
            st.dataframe(data)
        # Xử lí data loại bỏ biến UNKNOWN
        elif data_name == "Data xử lí biến UNKNOWN và cân bằng dữ liệu": 
            st.text_area(label= "Mô tả bộ dữ liệu ", height=250,
                value=
    """- Dữ liệu được loại bỏ các biến UNKNOWN hoặc tương tự\n
- Loại bỏ các cột trên 70% null\n
- Loại bỏ tất cả các dòng chứa giá trị Nan\n
- Loại bỏ các cột chỉ chứa một giá trị\n
- Chuyển thuộc tính chữ thành số sử dụng Label Encoder\n
- Cân bằng dữ liệu\n
- Sau khi xử lí, dữ liệu có dòng và cột""")
            unk = ["UNKNOWN",
                    "CLIENT DIDN'T ANSWER", 
                    "CLIENT DID NOT ANSWER", 
                    "UNKNOWN RACE", 
                    "UNKNOWN EMPLOYMENT STATUS", 
                    "NOT APPLICABLE", 
                    "NOT APPLICABLE",
                    "UNKNOWN EMPLOYMENT HOURS"]
            # Thay thế giá trị UNKNOWN hoặc tương tự thành Nan
            data.replace(to_replace=unk, value=np.nan, inplace=True)
            # Loại bỏ các cột có trên 70% Nan
            for col in data.columns:
                if data[col].isnull().sum() > 0.7*len(data):
                    data.drop(col, axis=1, inplace=True)

            # Bỏ tất cả các dòng chứa giá trị Nan
            data.dropna(inplace=True)

            # Bỏ các cột chỉ chứa một giá trị sau khi xử lí
            for col in data.columns:
                if len(data[col].unique()) <= 1:
                    data.drop(col, axis=1, inplace=True)
            # Chuyển các thuộc tính chữ thành số bằng Label Encoder
            for col in data.columns:
                if data[col].dtypes != "int64":
                    data[col]= label_encoder.fit_transform(data[col])
            # Cân bằng dữ liệu
            X = data.drop("Stroke", axis = 1)
            y = data["Stroke"]
            smt = SMOTE()
            X_res_smt, y_res_smt = smt.fit_resample(X, y)
            data_res_smt = X_res_smt
            data_res_smt["Stroke"] = y_res_smt
            data = data_res_smt
            st.dataframe(data)
            st.text("Sau khi xử lí, dữ liệu có số dòng và cột" +  str(data.shape))
        else:
            st.text_area(label= "Mô tả bộ dữ liệu ", height=250,
                value=
    """- Dữ liệu được loại bỏ các biến UNKNOWN hoặc tương tự\n
- Loại bỏ các cột trên 70% null\n
- Loại bỏ tất cả các dòng chứa giá trị Nan\n
- Loại bỏ các cột chỉ chứa một giá trị\n
- Chuyển thuộc tính chữ thành số sử dụng Label Encoder\n
- Chọn lọc ra các biến có hệ số tương quan cao với biến mục tiêu (Stroke)\n
- Cân bằng dữ liệu""")
            unk = ["UNKNOWN",
                    "CLIENT DIDN'T ANSWER", 
                    "CLIENT DID NOT ANSWER", 
                    "UNKNOWN RACE", 
                    "UNKNOWN EMPLOYMENT STATUS", 
                    "NOT APPLICABLE", 
                    "NOT APPLICABLE",
                    "UNKNOWN EMPLOYMENT HOURS"]
            # Thay thế giá trị UNKNOWN hoặc tương tự thành Nan
            data.replace(to_replace=unk, value=np.nan, inplace=True)
            # Loại bỏ các cột có trên 70% Nan
            for col in data.columns:
                if data[col].isnull().sum() > 0.7*len(data):
                    data.drop(col, axis=1, inplace=True)

            # Bỏ tất cả các dòng chứa giá trị Nan
            data.dropna(inplace=True)

            # Bỏ các cột chỉ chứa một giá trị sau khi xử lí
            for col in data.columns:
                if len(data[col].unique()) <= 1:
                    data.drop(col, axis=1, inplace=True)
            # Chuyển các thuộc tính chữ thành số bằng Label Encoder
            for col in data.columns:
                if data[col].dtypes != "int64":
                    data[col]= label_encoder.fit_transform(data[col])
            
            # Cân bằng dữ liệu
            X = data.drop("Stroke", axis = 1)
            y = data["Stroke"]
            smt = SMOTE()
            X_res_smt, y_res_smt = smt.fit_resample(X, y)
            data_res_smt = X_res_smt
            data_res_smt["Stroke"] = y_res_smt
            data = data_res_smt
            # Lấy ra các thuộc tính có ảnh hưởng đến biến đích
            data_res_smt_corr = data.corr()
            correlation_index_SMT = data_res_smt_corr[((data_res_smt_corr["Stroke"] > 0.2) | (data_res_smt_corr["Stroke"] < -0.2))]["Stroke"].index
            data = data[correlation_index_SMT]
            st.dataframe(data)
            st.text("Sau khi xử lí, dữ liệu có số dòng và cột " +  str(data.shape))
        X = data.drop("Stroke", axis = 1)
        y = data["Stroke"]
        if train_size == 0.0:
            train_size = 0.8
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size)
        model.fit(X_train, y_train)
        predict = model.predict(X_test)
        st.text(classification_report(y_test.values, predict))
        sns.heatmap(confusion_matrix(y_test.values, predict), cmap="Blues", annot=True)
        st.pyplot()

        

Overwriting app.py


In [2]:
data = pd.read_csv(r"D:\Đại học\Năm 3\Kỳ 1\Phân tích và trực quan dữ liệu\Đồ án\Data\patient-characteristics-survey-pcs-2017-1.csv")

In [3]:
data

,Survey Year,Program Category,Region Served,Age Group,Sex,Transgender,Sexual Orientation,Hispanic Ethnicity,Race,Living Situation,...,Unknown Insurance Coverage,No Insurance,Medicaid Insurance,Medicaid Managed Insurance,Medicare Insurance,Private Insurance,Child Health Plus Insurance,Other Insurance,Criminal Justice Status,Three Digit Residence Zip Code
0,2017,OUTPATIENT,CENTRAL NY REGION,ADULT,FEMALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",WHITE ONLY,PRIVATE RESIDENCE,...,NO,NO,YES,YES,NO,NO,NO,NO,NO,132
1,2017,SUPPORT,WESTERN REGION,ADULT,FEMALE,"NO, NOT TRANSGENDER",UNKNOWN,"NO, NOT HISPANIC/LATINO",WHITE ONLY,PRIVATE RESIDENCE,...,NO,NO,YES,UNKNOWN,YES,NO,NO,NO,UNKNOWN,146
2,2017,OUTPATIENT,CENTRAL NY REGION,ADULT,FEMALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",WHITE ONLY,PRIVATE RESIDENCE,...,NO,NO,YES,NO,NO,NO,NO,NO,NO,131
3,2017,OUTPATIENT,NEW YORK CITY REGION,ADULT,FEMALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",WHITE ONLY,PRIVATE RESIDENCE,...,NO,NO,YES,NO,YES,NO,NO,NO,NO,112
4,2017,OUTPATIENT,CENTRAL NY REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",WHITE ONLY,PRIVATE RESIDENCE,...,YES,NO,NO,NOT APPLICABLE,NO,NO,NO,NO,NO,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187187,2017,RESIDENTIAL,WESTERN REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"YES, HISPANIC/LATINO",BLACK ONLY,OTHER LIVING SITUATION,...,NO,NO,YES,YES,NO,NO,NO,NO,NO,142
187188,2017,OUTPATIENT,WESTERN REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",BLACK ONLY,OTHER LIVING SITUATION,...,NO,NO,YES,YES,YES,NO,NO,NO,YES,142
187189,2017,EMERGENCY,WESTERN REGION,ADULT,FEMALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",BLACK ONLY,OTHER LIVING SITUATION,...,NO,NO,YES,YES,NO,NO,NO,NO,YES,888
187190,2017,RESIDENTIAL,NEW YORK CITY REGION,ADULT,FEMALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"YES, HISPANIC/LATINO",MULTI-RACIAL,OTHER LIVING SITUATION,...,NO,NO,YES,NO,NO,NO,NO,NO,NO,104
